In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import ttk  # Import ttk for Combobox
from PIL import ImageTk, Image
import numpy as np
import pyttsx3  # Import the text-to-speech library
from keras.models import load_model
from googletrans import Translator
import cv2  # Import OpenCV for real-time video capture
from threading import Thread

# Load the trained model to classify sign
model = load_model('model30.h5')

# Dictionary to label all traffic signs class.
classes = { 1:'Speed limit (20km/h)', 2:'Speed limit (30km/h)', 3:'Speed limit (50km/h)',
            4:'Speed limit (60km/h)', 5:'Speed limit (70km/h)', 6:'Speed limit (80km/h)',
            7:'End of speed limit (80km/h)', 8:'Speed limit (100km/h)', 9:'Speed limit (120km/h)',
           10:'No passing', 11:'No passing veh over 3.5 tons', 12:'Right-of-way at intersection',
           13:'Priority road', 14:'Yield', 15:'Stop', 16:'No vehicles', 17:'Veh > 3.5 tons prohibited',
           18:'No entry', 19:'General caution', 20:'Dangerous curve left', 21:'Dangerous curve right',
           22:'Double curve', 23:'Bumpy road', 24:'Slippery road', 25:'Road narrows on the right',
           26:'Road work', 27:'Traffic signals', 28:'Pedestrians', 29:'Children crossing',
           30:'Bicycles crossing', 31:'Beware of ice/snow', 32:'Wild animals crossing',
           33:'End speed + passing limits', 34:'Turn right ahead', 35:'Turn left ahead',
           36:'Ahead only', 37:'Go straight or right', 38:'Go straight or left', 39:'Keep right',
           40:'Keep left', 41:'Roundabout mandatory', 42:'End of no passing',
           43:'End no passing veh > 3.5 tons' }

# Initialize GUI
top = tk.Tk()
top.geometry('1200x800')  # Increased width to accommodate video feed
top.title('Traffic Sign Classification')
top.configure(bg='#f7f7f7')

# Initialize the text-to-speech engine
engine = pyttsx3.init()
translator = Translator()

# Global variable to store the classified sign text
classified_sign_text = ""

def classify(image):
    global classified_sign_text
    image = image.resize((30, 30))
    image = np.expand_dims(image, axis=0)
    image = np.array(image)
    
    # Use model.predict and np.argmax to get the predicted class
    pred = np.argmax(model.predict(image), axis=1)[0]
    classified_sign_text = classes[pred + 1]  # Store the classified sign text
    update_display()  # Update the display with the classified text

def translate_text(text, dest_lang):
    try:
        translation = translator.translate(text, dest=dest_lang)
        return translation.text
    except Exception as e:
        messagebox.showerror("Translation Error", str(e))
        return text

def speak():
    if classified_sign_text:
        translated_text = translate_text(classified_sign_text, language_var.get())
        engine.say(translated_text)
        engine.runAndWait()

def update_display():
    if classified_sign_text:
        # Display the classified sign text in English by default
        label.configure(text=classified_sign_text)

def update_translation():
    if classified_sign_text:
        translated_text = translate_text(classified_sign_text, language_var.get())
        label.configure(text=translated_text)

def process_frame(frame):
    # Preprocess frame for model prediction
    image = Image.fromarray(frame)
    classify(image)

def start_camera():
    def capture_video():
        cap = cv2.VideoCapture(0)
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert color from BGR to RGB
            process_frame(frame)  # Process the frame
            
            # Update GUI
            image = Image.fromarray(frame)
            im = ImageTk.PhotoImage(image)
            sign_image.configure(image=im)
            sign_image.image = im
            top.update_idletasks()
        cap.release()
    
    # Start the video capture in a separate thread
    thread = Thread(target=capture_video)
    thread.daemon = True
    thread.start()

def translate_button_clicked():
    update_translation()

# Create Frames for a better layout
header_frame = tk.Frame(top, bg='#4A90E2', padx=20, pady=20)
header_frame.pack(fill='x')

content_frame = tk.Frame(top, bg='#f7f7f7')
content_frame.pack(fill='both', expand=True, padx=20, pady=20)

image_frame = tk.Frame(content_frame, bg='#ffffff', borderwidth=2, relief='solid')
image_frame.pack(side='left', fill='both', expand=True)

button_frame = tk.Frame(content_frame, bg='#f7f7f7')
button_frame.pack(side='right', fill='y')

# Header
heading = tk.Label(header_frame, text="Traffic Sign Classification", font=('Arial', 28, 'bold'), bg='#4A90E2', fg='#ffffff')
heading.pack()

# Upload button
upload = tk.Button(button_frame, text="Upload Image", command=lambda: upload_image(), bg='#4A90E2', fg='#ffffff', font=('Arial', 12, 'bold'), width=20, height=2)
upload.pack(pady=10)

# New button for speaking the classified sign
speak_button = tk.Button(button_frame, text="Speak", command=lambda: speak(), bg='#4A90E2', fg='#ffffff', font=('Arial', 12, 'bold'), width=20, height=2)
speak_button.pack(pady=10)

# New button for translating the sign
translate_button = tk.Button(button_frame, text="Translate", command=translate_button_clicked, bg='#4A90E2', fg='#ffffff', font=('Arial', 12, 'bold'), width=20, height=2)
translate_button.pack(pady=10)

# New button for starting the camera feed
camera_button = tk.Button(button_frame, text="Start Camera", command=lambda: start_camera(), bg='#4A90E2', fg='#ffffff', font=('Arial', 12, 'bold'), width=20, height=2)
camera_button.pack(pady=10)

# Language selection dropdown
language_var = tk.StringVar(value='en')
language_menu = ttk.Combobox(button_frame, textvariable=language_var, values=['en', 'es', 'fr', 'de', 'zh-cn', 'ja', 'ko', 'ar'], state='readonly')
language_menu.set('Select Language')
language_menu.config(width=20, font=('Arial', 12, 'bold'))
language_menu.pack(pady=10)

# Bind language selection to update the translation if needed
language_menu.bind('<<ComboboxSelected>>', lambda event: update_translation())

# Image display and label
sign_image = tk.Label(image_frame, bg='#ffffff')
sign_image.pack(pady=20)

label = tk.Label(image_frame, text='', font=('Arial', 18, 'bold'), bg='#ffffff')
label.pack(pady=10)

def upload_image():
    try:
        file_path = filedialog.askopenfilename()
        uploaded = Image.open(file_path)
        uploaded.thumbnail(((image_frame.winfo_width() / 1.5), (image_frame.winfo_height() / 1.5)))
        im = ImageTk.PhotoImage(uploaded)
        
        sign_image.configure(image=im)
        sign_image.image = im
        classify(uploaded)
    except Exception as e:
        messagebox.showerror("File Error", str(e))

top.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━